In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tempfile

import os
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import MeanAbsoluteError, MeanSquaredError
from tensorflow.keras.optimizers import Adagrad, Adadelta, Adam, Adamax, Ftrl, Nadam, SGD, RMSprop 
from tensorflow.keras.layers import Bidirectional, Dense, LSTM, GRU, Conv1D, GlobalAveragePooling1D, Lambda, Flatten, Dropout, Embedding, Input
from tqdm import tqdm
matplotlib.style.use("seaborn-whitegrid")
pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)
plt.rcParams["figure.figsize"] = (15, 10)

%config InlineBackend.figure_format = 'retina'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.simplefilter('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
# Detect hardware
try:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu_address)  # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    # Going back and forth between TPU and host is expensive.
    # Better to run 128 batches on the TPU before reporting back.
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    print("Number of accelerators: ", strategy.num_replicas_in_sync)
except ValueError:
    print('TPU failed to initialize.')


Running on TPU  ['10.105.239.250:8470']
Number of accelerators:  8


In [ ]:
import pdb

In [ ]:
!gdown 12ceraaz41xJ503VhZlU-WnBB5pR8QWWf
!gdown 1mocZNvYWzWL9U-kygm9QU4ejuJoO0jyo

Downloading...
From: https://drive.google.com/uc?id=12ceraaz41xJ503VhZlU-WnBB5pR8QWWf
To: /content/train.feather
100% 12.2M/12.2M [00:00<00:00, 65.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mocZNvYWzWL9U-kygm9QU4ejuJoO0jyo
To: /content/test.feather
100% 45.8k/45.8k [00:00<00:00, 50.9MB/s]


In [ ]:
df = pd.read_feather(
    '/content/train.feather'
)
df = df.drop('data_source', axis=1)
df = df.fillna(0)
train = df.sample(frac=0.8)
val = df.drop(train.index, axis=0)

In [ ]:
max_length = 2250
trunc_type = 'post'
embedding_dim = 64

In [ ]:
def split_seq(df):
    sentences = np.array([
        df[['protein_sequence']].to_numpy()[:, 0],
    ])
    splitted = []
    for i in sentences[0, :]:
        splitted.append(list(i))

    return np.array(splitted)

train_seq, test_seq = split_seq(train), split_seq(val)

train_ph, val_ph = train['pH'].to_numpy().reshape(
    (len(train), 1)), val['pH'].to_numpy().reshape((len(val), 1))
train_tm, val_tm = train['tm'].to_numpy().reshape(
    (len(train), 1)), val['tm'].to_numpy().reshape((len(val), 1))


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_seq)
sequences = tokenizer.texts_to_sequences(train_seq)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
vocab_size = len(tokenizer.word_index)
train = np.append(np.append(padded, train_ph, 1), train_tm, 1)

testing_sequences = tokenizer.texts_to_sequences(test_seq)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)
val = np.append(np.append(testing_padded, val_ph, 1), val_tm, 1)


In [ ]:
length = len(train_seq)+len(test_seq)
df = tf.data.Dataset.from_tensor_slices(np.vstack([train,val]))

In [ ]:
# os.mkdir('/content/sequences')
dirname = '/content/sequences'
for ind, i in enumerate(df):
  example_path = os.path.join(dirname, f"sequence_{ind}.tfrecords")
  np.random.seed(0)
  with tf.io.TFRecordWriter(example_path) as file_writer:
    i = i.numpy()
    sequence, ph, tm = i[:-2], i[-2], i[-1]

    record_bytes = tf.train.Example(features=tf.train.Features(feature={
        "sequence": tf.train.Feature(float_list=tf.train.FloatList(value=sequence)),
        "ph": tf.train.Feature(float_list=tf.train.FloatList(value=[ph])),
        "tm": tf.train.Feature(float_list=tf.train.FloatList(value=[tm])),
    })).SerializeToString()
    file_writer.write(record_bytes)

In [ ]:
filenames = tf.io.gfile.glob('/content/sequences/*.tfrecords')
split = int(len(filenames) * 0.2)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]

In [ ]:
t = tf.data.Dataset.from_tensor_slices(filenames)
for i in t:
  tf.strings.unicode_decode(i, 'UTF-8').to_list()

AttributeError: ignored

> /usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py(513)__getattr__()
    511         from tensorflow.python.ops.numpy_ops import np_config
    512         np_config.enable_numpy_behavior()""".format(type(self).__name__, name))
--> 513     self.__getattribute__(name)
    514 
    515   @staticmethod

ipdb> exit


In [ ]:
with open('/content/sequences/sequence_0.tfrecords', 'rb') as f:
  f.read()

b'a#\x00\x00\x00\x00\x00\x00m\x99I\xd2\n\xdeF\n\x0e\n\x02tm\x12\x08\x12\x06\n\x04\x9a\x99;B\n\x0e\n\x02ph\x12\x08\x12\x06\n\x04\x00\x00\xe0@\n\xbbF\n\x08sequence\x12\xaeF\x12\xabF\n\xa8F\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [ ]:
get_training_dataset()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 2] No such file or directory: '/tmp/__autograph_generated_filei8w4esbp.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 2] No such file or directory: '/tmp/__autograph_generated_filei8w4esbp.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


OperatorNotAllowedInGraphError: ignored

> <ipython-input-145-4eac9d3d460e>(5)read_tfrecord()
      3 @tf.function
      4 def read_tfrecord(example):
----> 5   for i in example:
      6     path = i
      7   with open(path, "rb") as local_file: # <= change here

ipdb> n


In [ ]:
BATCH_SIZE = 128

@tf.function
def read_tfrecord(example):
  for i in example:
    path = i
  with open(path, "rb") as local_file: # <= change here
      seq = local_file.read()
  sequence, ph, tm = example[:-2], example[-2], example[-1]
  return sequence, ph, tm


def load_dataset(filenames):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    print(dataset)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset,
                                 cycle_length=16,
                                 num_parallel_calls=AUTO)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
    return dataset

def get_batched_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False) # drop_remainder will be needed on TPU
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset


def get_training_dataset():
    dataset = get_batched_dataset(training_filenames)
    dataset = strategy.experimental_distribute_dataset(dataset, tf.distribute.InputOptions(
            experimental_fetch_to_device=False))
    return dataset


def get_validation_dataset():
    dataset = get_batched_dataset(validation_filenames)
    dataset = strategy.experimental_distribute_dataset(dataset, tf.distribute.InputOptions(
            experimental_fetch_to_device=False))
    return dataset

def concat(input):
    # if len(input[0].shape) >=2:
    #   input[1] = tf.tile(tf.reshape(input[1], (-1, 1, 1)), (1, 1, input[0].shape[-1]))
    #   return tf.keras.layers.concatenate([input[0], input[1]], axis=1)
    # else:
    return tf.keras.layers.concatenate([input[0], input[1]], axis=1)

In [ ]:
# %pdb
class MyModel2(tf.keras.Model):
  def __init__(self):
    super(MyModel2, self).__init__()
    self.emb_dim, self.dropout_rate = 2250, 0.2
    self.input_ph = tf.keras.layers.Input((1,))
    self.input_seq = tf.keras.layers.Input((2251, ))
    self.seq_layers =  [] 
    self.seq_layers.extend([
                        Embedding(vocab_size + 1, self.emb_dim, name='embedding'),
                        Conv1D(filters=64, kernel_size=5, activation='relu'),
                        Dropout(self.dropout_rate),  
                        Conv1D(filters=64, kernel_size=3, activation='relu'),
                        Dropout(self.dropout_rate),
                        Bidirectional(LSTM(128, return_sequences=True, name='LSTM_5')),
                        Dropout(self.dropout_rate),
                        Bidirectional(LSTM(32, return_sequences=True, name='LSTM_7')),
                        Dropout(self.dropout_rate),
                        Bidirectional(LSTM(16, return_sequences=True)),
                        GlobalAveragePooling1D(),
                        ])

    self.ph_layers = [ 
                      tf.keras.layers.Dense(32, name='dense3_ph', activation='relu'),
                      tf.keras.layers.Dropout(self.dropout_rate),  
                      tf.keras.layers.Dense(1, name='output_ph')
                      ]

    self.lambda_layer = Lambda(function=concat, name='lambda_layer')
    self.flatten = Flatten(name='flatten')
    self.dense_combined = Dense(32, activation='relu', name='dense_combined')
    self.lambda_helper = Lambda(lambda x: tf.reshape(x, (-1, 32, 1)))
    self.last_dense = Dense(1, activation='relu', name='output')

  def call(self, inputs):
    seq, ph = inputs

    for layer in self.seq_layers:
      seq = layer(seq)

    for layer in self.ph_layers:
      ph = layer(ph)

    x = self.lambda_layer([seq, ph])
    x = self.flatten(x)
    x = self.dense_combined(x)
    x = self.lambda_helper(x)
    x = self.last_dense(x)
    return x

In [ ]:
with strategy.scope():
    model = MyModel2()
    loss_object = MeanAbsoluteError(
        reduction=tf.keras.losses.Reduction.NONE)

    def compute_loss(labels, predictions):
        per_example_loss = loss_object(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss,
                                          global_batch_size=BATCH_SIZE *
                                          strategy.num_replicas_in_sync)
    test_loss = MeanAbsoluteError()
    optimizer = tf.keras.optimizers.Adam()

    @tf.function
    def distributed_train_step(dataset_inputs):
        per_replica_losses = strategy.run(train_step, args=(dataset_inputs, ))
        print(per_replica_losses)
        return strategy.reduce(tf.distribute.ReduceOp.SUM,
                               per_replica_losses,
                               axis=None)

    @tf.function
    def distributed_test_step(dataset_inputs):
        strategy.run(test_step, args=(dataset_inputs, ))

    def train_step(inputs):
        sequence, ph, tm = inputs

        with tf.GradientTape() as tape:
            predictions = model([sequence, ph])
            loss = compute_loss(tm, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        return loss

    def test_step(inputs):
        sequence, ph, tm = inputs

        predictions = model([sequence, ph])
        loss = loss_object(tm, predictions)



In [ ]:
EPOCHS = 40
with strategy.scope():
  for _ in range(EPOCHS):
    total_loss = 0.0
    num_batches = 0
    for x in get_training_dataset():
      total_loss += distributed_train_step(x)
      num_batches += 1
    train_loss = total_loss / num_batches

    # TESTING LOOP
    for x in get_validation_dataset():
      distributed_test_step(x)

    test_loss.result()
    test_loss.reset_states()


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 2] No such file or directory: '/tmp/__autograph_generated_fileugp1j_q2.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 2] No such file or directory: '/tmp/__autograph_generated_fileugp1j_q2.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


UnimplementedError: ignored